# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv("/datasets/users_behavior.csv")

In [3]:
display(df.head(10))

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

Как видно из первой десятки, данные состоят из двух категорий: признаки и целевой признак. Количество звонков, минут, сообщений и трафика - это признак. А присутствия тарифа Ultra, которое отображается 0/1, является классификацией и целевым признаком.

## Разбейте данные на выборки

Для создания модели и дальнейшей ее валидации, нам понадобится разделить данные на три группы: тренировочная, валидационная и тестировочная. Они будут разделены в соотношении 3:1:1 или 60%, 20% и 20%. 

In [5]:
features = df.drop(["is_ultra"], axis = 1)
target = df["is_ultra"]

In [6]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=12345)

Проверяем соотношение: 

In [7]:
print(len(features_train)/len(features)*100)
print(len(features_valid)/len(features)*100)
print(len(features_test)/len(features)*100)

59.98755444928439
20.00622277535781
20.00622277535781


## Исследуйте модели

Стоит проверить каждую из трех моделей: Решающее дерево, случайный лес и логистическая регрессия. 

Однако, у всех моделей есть разные гиперпараметры которые можно менять. Чтобы найти лучшую, будет проведена работа по поиску лучшей модели для каждого типа. Результаты будут сравниваться с валидационными данными и выбираться лучшая. 

In [8]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 21):
    model = DecisionTreeClassifier(random_state=12345,max_depth=depth,criterion="gini")
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid)
    result = model.score(features_valid,target_valid)
    if result > best_result:
        best_dtcg_model = model
        best_result = result
        best_depth = depth

In [9]:
print("Точность лучшей модели:", best_result)
print("Глубина лучшей модели:", best_depth)

Точность лучшей модели: 0.7853810264385692
Глубина лучшей модели: 3


In [10]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 21):
    model = DecisionTreeClassifier(random_state=12345,max_depth=depth,criterion="entropy")
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid)
    result = model.score(features_valid,target_valid)
    if result > best_result:
        best_dtce_model = model
        best_result = result
        best_depth = depth

In [11]:
print("Точность лучшей модели:", best_result)
print("Глубина лучшей модели:", best_depth)

Точность лучшей модели: 0.7853810264385692
Глубина лучшей модели: 3


In [12]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 101, 5):
    for depth in range (1, 51, 5):
        model = RandomForestClassifier(random_state=12345,n_estimators=est, max_depth=depth,criterion="gini")
        model.fit(features_train,target_train)
        predictions_valid = model.predict(features_valid)
        result = model.score(features_valid,target_valid)
        if result > best_result:
            best_rfcg_model = model
            best_result = result
            best_depth = depth
            best_est = est

In [13]:
print("Точность лучшей модели:", best_result)
print("Глубина лучшей модели:", best_depth)
print("Число деревьев лучшей модели:", best_est)

Точность лучшей модели: 0.80248833592535
Глубина лучшей модели: 6
Число деревьев лучшей модели: 31


In [14]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 101, 5):
    for depth in range (1, 51, 5):
        model = RandomForestClassifier(random_state=12345,n_estimators=est, max_depth=depth,criterion="entropy")
        model.fit(features_train,target_train)
        predictions_valid = model.predict(features_valid)
        result = model.score(features_valid,target_valid)
        if result > best_result:
            best_rfce_model = model
            best_result = result
            best_depth = depth
            best_est = est

In [15]:
print("Точность лучшей модели:", best_result)
print("Глубина лучшей модели:", best_depth)
print("Число деревьев лучшей модели:", best_est)

Точность лучшей модели: 0.807153965785381
Глубина лучшей модели: 6
Число деревьев лучшей модели: 21


In [16]:
best_model = None
best_result = 0
best_itter = 0
best_est = 0
for strght in range(1, 10):
    for itter in range (100, 2000,100):
        model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=itter, C=strght)
        model.fit(features_train,target_train)
        predictions_valid = model.predict(features_valid)
        result = model.score(features_valid,target_valid)
        if result > best_result:
            best_lr_model = model
            best_result = result
            best_itter = itter
            best_strght = strght

In [17]:
print("Точность лучшей модели:", best_result)
print("Иттерация лучшей модели:", best_itter)
print("Число силы регуляризации лучшей модели:", best_strght)

Точность лучшей модели: 0.7558320373250389
Иттерация лучшей модели: 100
Число силы регуляризации лучшей модели: 5


Из проведенных анализов видно, что лучше всего справился случайный лес с глубинной 8 и числе деревьев 40, и значением "criterion" на "entropy". 

Стоит отметить, что решающее дерево тоже достигло результата больше 75% верных ответов, но в сравнении с решающим лесом она работает намного быстрее. Поэтому, выкидывать эту модель не стоит.

## Проверьте модель на тестовой выборке

Теперь стоит проверить на тестовой выборке, правда ли результат больше 75%. 

In [18]:
test_result = best_rfce_model.score(features_test,target_test)

In [19]:
print("Точность тестовой выборки:",test_result)

Точность тестовой выборки: 0.7978227060653188


Как видно из результатов, модель набрала больше 75%, что делает ее пригодной для работы. 

## (бонус) Проверьте модели на адекватность

Чтобы проверить нашу модель на адекватность, мы сравним ее с "дурачковой" моделью. 

In [21]:
dummy = DummyClassifier()
dummy.fit(features_train,target_train)
dummy_result = dummy.score(features_test,target_test)

In [22]:
print("Точность 'дурачковой' выборки:",dummy_result)

Точность 'дурачковой' выборки: 0.6842923794712286


Как видно из результатов, наша модель имеет более 10% преимущество перед "дурачковой" моделью, что говорит о ее адекватности. 

# Заключение

Из проделанной работы, было выявлено, что лучшей моделью для предсказания тарифа является случайный лес с глубинной 8 и числе деревьев 40, и значением "criterion" на "entropy". В данном проекте модель располагается под переменной "best_rfce_model". 

Так же, тест на адекватность был "сдан" с более чем 10% преимущества перед "дурачковой" моделью.

Однако, стоить отметить, что все другие модели с правильными гиперпараметрами достигли более 75% точности на стадии тренировки. Что значит, что могут быть и использованы другие модели в зависимости от задачи. 